# Part 2

In [76]:
#Imports Section

import networkx as nx
import numpy as np
from tqdm import tqdm
import netwulf

## Exercise 1

In [114]:
#Defining the karateclub data as a function

karclub_data = nx.karate_club_graph()

In [116]:
#Setting the color of each node based on the club split

karclub_data
club_membership = nx.get_node_attributes(karclub_data, "club")

In [123]:
#setting the colors of different club nodes
colors = {}
clubs = []#For use later
mrhi_members = []
officer_members = []

for node in range(len(karclub_data.nodes)):
    if club_membership[node] == 'Mr. Hi': 
        colors[node] = "blue"
        mrhi_members.append(node)
    else:
        colors[node] = "red"
        officer_members.append(node)

clubs.append(mrhi_members)
clubs.append(officer_members)
nx.set_node_attributes(karclub_data,colors,"color")

In [111]:
#Plotting the karate club datases
netwulf.interactive.visualize(karclub_data,config={"Node color": "color"})

(None, None)

## Exercise 2

In [126]:
#Modularity function

def Modularity(graph,communities):
    # Inputs : Networkx Graph
    # Output : Modularity of given Networkx Graph
    m = graph.number_of_edges()
    Q = 0
    for community in communities:
        E = 0
        A = 0
        for node in community:
            E +=graph.degree[node]
            for neighbor in graph.neighbors(node):
                if neighbor in community:
                    A += 1
        Q += (A - (E * E) / (2 * m)) / (2 * m)
    return Q
    
    return modularity

In [127]:
Modularity(karclub_data,clubs)

0.3582347140039448